## V3Det

In [1]:
import os
import torch

from pathlib import Path
from mmdet.apis import DetInferencer
from IPython.utils import io
from pyarrow.feather import write_feather, read_feather


wdir = Path('/home/yu/OneDrive/Construal')
model_dir = Path('/home/yu/OneDrive/Construal/pretrained-models/v3det')
os.chdir(model_dir)

In [30]:
# Initialize the DetInferencer
inferencer = DetInferencer(
    # DINO
    # model='/home/yu/OneDrive/Construal/pretrained-models/v3det/checkpoints/configs/v3det/dino-4scale_swin_16xb1_sample1e-3_v3det_36e.py', 
    # weights='/home/yu/OneDrive/Construal/pretrained-models/v3det/checkpoints/DINO_V3Det_SwinB.pth',

    # DETR (DETR is faster)
    model='/home/yu/OneDrive/Construal/pretrained-models/v3det/checkpoints/configs/v3det/deformable-detr-refine-twostage_swin_16xb2_sample1e-3_v3det_50e.py',
    weights='/home/yu/OneDrive/Construal/pretrained-models/v3det/checkpoints/Deformable_DETR_V3Det_SwinB.pth',

    device='cuda:1'
)

Loads checkpoint by local backend from path: /home/yu/OneDrive/Construal/pretrained-models/v3det/checkpoints/Deformable_DETR_V3Det_SwinB.pth
10/26 04:06:43 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/yu/Software/python/python-env/py311-mmdet/lib/python3.11/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [31]:
pjson = read_feather('/home/yu/OneDrive/Construal/data/pjson.feather')
valid_pids = pjson.loc[(pjson.category=='Accessories') | (pjson.category=='Product Design'), 'pid'].to_list()

In [32]:
img_root = Path('/home/yu/chaoyang/research-resources/kickstart-raw-from-amrita/kickstarter-image')
# img_paths = list(img_paths.glob('**/profile_full.jpg'))


In [34]:
with io.capture_output() as captured:
    for i, pid in enumerate(valid_pids):
        # print progress
        if i % 100 == 0:
            print(f'Processing: {i}/{len(valid_pids)}')
        
        # construct image path
        img_path = img_root / f'{pid}/profile_full.jpg'
        if not img_path.exists():
            print(f'{img_path} does not exist')
            continue

        # inference
        out = inferencer(str(img_path), show=False)['predictions'][0]

        # save results
        torch.save(out, wdir/f'data/v3det/on-kickstarter/per-image-results/{pid}.pt')
    

**Randomly select 100 training images from V3Det**

In [13]:
import shutil
import random

In [ ]:
p = model_dir/'data/V3Det/images'
image_paths = random.sample(list(p.glob('**/*.jpg')), 100)

for i, path in enumerate(image_paths):
    # print(path.name)
    shutil.copy(path, wdir/f'data/sharing/v3det-image-examples/{path.name}')

## BigDetection

In [1]:
import mmcv
import os
from mmdet.apis import init_detector, inference_detector
from pathlib import Path

wdir = Path('/home/yu/OneDrive/Construal/pretrained-models/bigdet/bigdetection-main')
os.chdir(wdir)

/home/yu/Software/python/python-env/py39-mmdet-legacy/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


apex is not installed
apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [ ]:
tools/test.py configs/BigDetection/cbnetv2/htc_cbv2_swin_base_giou_4conv1f_adamw_bigdet.py \
    /home/yu/OneDrive/Construal/pretrained-models/bigdet/checkpoints/htc_cbv2_swin_base_giou_4conv1f_bigdet.pth \
        --eval bbox



In [ ]:
# Specify the path to model config and checkpoint file
config_file = '/home/yu/OneDrive/Construal/pretrained-models/bigdet/bigdetection-main/configs/BigDetection/cbnetv2/htc_cbv2_swin_base_giou_4conv1f_adamw_bigdet.py'
checkpoint_file = '/home/yu/OneDrive/Construal/pretrained-models/bigdet/checkpoints/htc_cbv2_swin_base_giou_4conv1f_bigdet.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
img = 'data/V3Det/images/a00000066/0_2530_11591900204_c1c10c1531_c.jpg'  # or img = mmcv.imread(img), which will only load it once
result = inference_detector(model, img)